In [ ]:
import numpy as np

# defining the position of the plate corner points
pos = np.array(
    [
        [0.0, 0.0, 0.0],
        [1.54503268, 4.1300398, 7.26136495],
        [-0.01769469, 3.9841196, 8.49703339],
        [-0.23864989, 3.14708537, 9.81674623],
        [-0.38529401, 1.96770111, 10.60735655],
        [-0.45841928, 0.66695416, 10.94897378],
        [-0.45841928, -0.66695416, 10.94897378],
        [-0.38529401, -1.96770111, 10.60735655],
        [-0.23864989, -3.14708537, 9.81674623],
        [-0.01769469, -3.9841196, 8.49703339],
        [1.54503268, -4.1300398, 7.26136495],
        [1.71039665, -3.97159687, 8.49204017],
        [2.01066217, -3.12943185, 9.78719788],
        [2.118729, -1.95450565, 10.55854696],
        [2.16733995, -0.66340879, 10.88984164],
        [2.16733995, 0.66340879, 10.88984164],
        [2.118729, 1.95450565, 10.55854696],
        [2.01066217, 3.12943185, 9.78719788],
        [1.71039665, 3.97159687, 8.49204017],
        [0.86307674, 4.1565, 7.42381981],
        [0.86307674, -4.1565, 7.42381981],
        [0.17884119, 0.0, 0.93301438],
        [0.41766592, 0.0, 2.00472644],
        [0.47651992, 0.51570164, 2.41872723],
        [1.01186906, 0.85948465, 4.67117852],
        [1.44706777, 2.71993277, 7.35496513],
        [1.53424698, 1.34379336, 7.83340153],
        [0.47651992, -0.51570164, 2.41872723],
        [1.01186906, -0.85948465, 4.67117852],
        [1.44706777, -2.71993277, 7.35496513],
        [1.53424698, -1.34379336, 7.83340153],
        [-0.06579256, 1.3268467, 5.5315951],
        [-0.04266184, 2.05530303, 7.25510078],
        [-0.09211119, 1.26740889, 7.82771638],
        [-0.06579256, -1.3268467, 5.5315951],
        [-0.04266184, -2.05530303, 7.25510078],
        [-0.09211119, -1.26740889, 7.82771638],
    ]
)

# Defining constants (should not change)
plate_point_indices = [
    [19, 2, 18, 1],
    [2, 3, 17, 18],
    [3, 4, 16, 17],
    [4, 5, 15, 16],
    [5, 6, 14, 15],
    [6, 7, 13, 14],
    [7, 8, 12, 13],
    [8, 9, 11, 12],
    [9, 20, 10, 11],
]
# extract_plate_point_indices()
# pos = np.load("TUDV3kite_plate_corner_points.npy")
A_projected = 19.49  # m^2
rho = 1.225  # kg/m^3

# Defining Variables (can change)
is_with_printing_plate_info = False
Va_norm = 10  # m/s
aoa = 10  # Angle of attack in degrees

Fa_plate_array = []

for i, indices in enumerate(plate_point_indices):
    # Defining vectors and points
    diagonal_1 = pos[indices[0]] - pos[indices[2]]
    diagonal_2 = pos[indices[1]] - pos[indices[3]]
    leading_edge = pos[indices[1]] - pos[indices[0]]
    trailing_edge = pos[indices[3]] - pos[indices[2]]
    middle_leading_edge_point = pos[indices[0]] + 0.5 * leading_edge
    middle_trailing_edge_point = pos[indices[2]] + 0.5 * trailing_edge
    chord_line = middle_trailing_edge_point - middle_leading_edge_point

    # Calculate normal vector
    # Because its assumed that aerodynamic force acts perpendicular to the plate
    normal_vec = np.cross(diagonal_1, diagonal_2)
    normal_unit_vec = normal_vec / np.linalg.norm(normal_vec)

    # Calculating angle between chord_line and x-axis
    x_axis = np.array([1, 0, 0])
    norm_x_axis = np.linalg.norm(x_axis)
    norm_chord_line = np.linalg.norm(chord_line)
    dot_product_divided_by_norms = np.dot(chord_line, x_axis) / (
        norm_chord_line * norm_x_axis
    )
    angle_with_airflow = np.arccos(dot_product_divided_by_norms)

    # Adjust AoA based on this angle
    aoa_corrected = aoa + angle_with_airflow

    # Calculate lift coefficient
    Cl_p = 2 * np.pi * np.sin(np.deg2rad(aoa_corrected))

    # Area calculation using Heron's formula (for a quadrilateral, split into two triangles)
    sides = [
        np.linalg.norm(pos[indices[i]] - pos[indices[(i + 1) % 4]]) for i in range(4)
    ]
    s = sum(sides) / 2
    area_p = np.sqrt((s - sides[0]) * (s - sides[1]) * (s - sides[2]) * (s - sides[3]))

    # Calculate plate aerodynamic force using the lift-equation
    # essentially, it is assumed here that Fa_plate = Lift_plate
    # this is not entirely correct as it has a different orientation
    # and drag is not separately calculated
    # nontheless for a simple fast approximation it works
    Fa_plate_norm = 0.5 * rho * Va_norm**2 * Cl_p * area_p

    Fa_plate = np.array(
        [
            Fa_plate_norm * normal_unit_vec[0],
            Fa_plate_norm * normal_unit_vec[1],
            Fa_plate_norm * normal_unit_vec[2],
        ]
    )

    Fa_plate_array.append(Fa_plate)

    if is_with_printing_plate_info:
        print(f" ")
        print(f"plate number: {i}")
        print(f"aoa: {aoa}, aoa_corrected: {aoa_corrected}")
        print(f"Plate Area: {area_p:.3f}m^2")
        print(f"Cl_p: {Cl_p}")
        print(f"Lift force of the plate: {Fa_plate_norm:.3f}N")

print(f"--------------------")
print(f"    Kite Results    ")
print(f"--------------------")

# Sum forces for total force components
Fa = np.sum(Fa_plate_array, axis=0)
print(f"Fa_x: {Fa[0]:.3f}N")
print(f"Fa_y: {Fa[1]:.3f}N")
print(f"Fa_z: {Fa[2]:.3f}N")

# Airflow direction is equal to the drag direction
Va = np.array([Va_norm * np.cos(np.deg2rad(aoa)), 0, Va_norm * np.sin(np.deg2rad(aoa))])
drag_direction = Va
drag_unit_vector = Va / np.linalg.norm(Va)

# Calculate lift-direction, which is perpendicular to airflow
# getting a vector perpendicular is done using the cross product
lift_direction = np.cross(Va, [0, 1, 0])
lift_unit_vector = lift_direction / np.linalg.norm(lift_direction)

# Side direction
side_unit_vector = np.array([0, -1, 0])

# Calculate Lift, Drag and SideForce
# using the dot product we find the components of Fa in the respective directions
Lift = np.dot(Fa, lift_unit_vector)
Drag = np.dot(Fa, drag_unit_vector)
Side = np.dot(Fa, side_unit_vector)
print(f"Lift: {Lift:.3f}N")
print(f"Drag: {Drag:.3f}N")
print(f"Side: {Side:.3f}N")

# Calculate coefficients
CL = Lift / (0.5 * rho * Va_norm**2 * A_projected)
CD = Drag / (0.5 * rho * Va_norm**2 * A_projected)
CS = Side / (0.5 * rho * Va_norm**2 * A_projected)
print(f" ")
print(f"AoA : {aoa:.3f}")
print(f"CL  : {CL:.3f}")
print(f"CD  : {CD:.3f}")
print(f"CS  : {CS:.3f}")

--------------------
    Kite Results    
--------------------
Fa_x: 32.653N
Fa_y: -0.000N
Fa_z: 1307.220N
Lift: 1281.691N
Drag: 259.154N
Side: 0.000N
 
AoA : 10.000
CL  : 1.074
CD  : 0.217
CS  : 0.000


In [ ]:
pos